<a href="https://colab.research.google.com/github/Tech-Scene/Tech-Scene.github.io/blob/main/OpenAI_Whisper_ar_ytdlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">
<h1>تفريغ الصوتيات باستخدم OpenAI Whisper</h1>

هذا الملف مبني على [هذا الملف](https://colab.research.google.com/gist/Kazuki-tam/04e85708e4fd1c4b8af180d317977f4d/whisper-mock-en.ipynb)


## 📖 كيفية الاستخدام
1. شغل خطوة "اﻹعداد".
2. اختر نمط التشغيل: يوتيوب أو ملف محلي.
  - إذا اخترت upload لتفريغ ملف تقوم برفعه بنفسك،  شغل خطوة اﻹعداد أولا ثم ارفع الملف إلى مجلد `download`.
  - إذا اخترت يوتيوب ضع رابط المقطع أو قائمة التشغيل في `user_input`
  - يمكنك تحديد بداية ونهاية قائمة التشغيل من الخيارين التاليين.
3. اختر لغة الملف.
4. اختر النموذج المستخدم في التفريغ، `large` يعطي نتائج أفضل لكن أبطأ.
5. شغل خطوة "التفريغ".
</div>

In [4]:
#@title اﻹعداد
!apt install libcublas11
!pip install --quiet --root-user-action=ignore --upgrade pip

from pathlib import Path

# Create folders
download_folder = Path("download")
if not download_folder.exists():
    download_folder.mkdir()
output_folder = Path("output")
if not output_folder.exists():
    output_folder.mkdir()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcublas11 is already the newest version (11.7.4.6~11.5.1-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
# Create the Drive folders
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

Mounted at /content/drive


In [11]:
# prompt: force delete /content/download/Audio

!rm -rf /content/drive


rm: cannot remove '/content/drive/.file-revisions-by-id': Operation canceled
rm: cannot remove '/content/drive/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.Trash-0': Directory not empty


In [12]:
# @title التفريغ
import mimetypes
from pathlib import Path

from google.colab import files

# These two functions are adapted from original whisper implementation
def format_timestamp(seconds, always_include_hours=False, decimal_marker="."):
    if seconds is not None:
      assert seconds >= 0, "non-negative timestamp expected"
      milliseconds = round(seconds * 1000.0)
      hours = milliseconds // 3_600_000
      milliseconds -= hours * 3_600_000
      minutes = milliseconds // 60_000
      milliseconds -= minutes * 60_000
      seconds = milliseconds // 1_000
      milliseconds -= seconds * 1_000
      hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
      return (
          f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
      )
    else:
        # we have a malformed timestamp so just return it as is
        return seconds

def write_srt(file, segments):
    for i, segment in enumerate(segments, start=1):
        if isinstance(segment, dict):
            segment_start = segment["start"]
            segment_end = segment["end"]
            segment_text = segment["text"]
        else:
            segment_start = segment.start
            segment_end = segment.end
            segment_text = segment.text
        start_time = format_timestamp(
            segment_start, always_include_hours=True, decimal_marker=","
        )
        end_time = format_timestamp(
            segment_end, always_include_hours=True, decimal_marker=","
        )
        file.write("%d\n" % i)
        file.write("%s --> %s\n" % (start_time, end_time))
        file.write(segment_text.strip().replace("-->", "->"))
        file.write("\n\n")

new_line_separators = {
    "White space ( )": " ",
    "Line feed (\\n)": "\n",
    "Carriage return + Line feed (\\r\\n)": "\r\n",
    "Carriage return (\\r)": "\r"
}

# options
converter = "faster-whisper"  # @param ["whisper", "faster-whisper", "whisper-jax", "insanely-fast-whisper"]
process_type = "upload"  # @param ["youtube", "google_drive", "upload", "direct_link", "curl"]
user_input = ""  # @param {type:"string"}
playlist_start = 1  # @param {type:"integer"}
playlist_end = 9999  # @param {type:"integer"}
language = "en"  # @param ["en", "ar", "Afrikaans", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Azerbaijani", "Bashkir", "Basque", "Belarusian", "Bengali", "Bosnian", "Breton", "Bulgarian", "Burmese", "Castilian", "Catalan", "Chinese", "Croatian", "Czech", "Danish", "Dutch", "English", "Estonian", "Faroese", "Finnish", "Flemish", "French", "Galician", "Georgian", "German", "Greek", "Gujarati", "Haitian", "Haitian Creole", "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hungarian", "Icelandic", "Indonesian", "Italian", "Japanese", "Javanese", "Kannada", "Kazakh", "Khmer", "Korean", "Lao", "Latin", "Latvian", "Letzeburgesch", "Lingala", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy", "Malay", "Malayalam", "Maltese", "Maori", "Marathi", "Moldavian", "Moldovan", "Mongolian", "Myanmar", "Nepali", "Norwegian", "Nynorsk", "Occitan", "Panjabi", "Pashto", "Persian", "Polish", "Portuguese", "Punjabi", "Pushto", "Romanian", "Russian", "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Sinhalese", "Slovak", "Slovenian", "Somali", "Spanish", "Sundanese", "Swahili", "Swedish", "Tagalog", "Tajik", "Tamil", "Tatar", "Telugu", "Thai", "Tibetan", "Turkish", "Turkmen", "Ukrainian", "Urdu", "Uzbek", "Valencian", "Vietnamese", "Welsh", "Yiddish", "Yoruba", "af", "am", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "iw", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]
model = "large-v2"  # @param ["large-v3", "large-v2", "medium", "base", "small", "tiny"]
new_line_separator = "Line feed (\\n)"  # @param {type:"string"} ["White space ( )", "Line feed (\\n)", "Carriage return + Line feed (\\r\\n)", "Carriage return (\\r)", " "]

# init
new_line_separator = new_line_separators.get(new_line_separator, " ")
is_whisper = converter == "whisper"
is_faster_whisper = converter == "faster-whisper"
is_whisper_jax = converter == "whisper-jax"
is_insanely_fast_whisper = converter == "insanely-fast-whisper"
mimetypes.init()
ydl_opts = {
    "format": "m4a/bestaudio/best",
    "postprocessors": [
        {  # Extract audio using ffmpeg
            "key": "FFmpegExtractAudio",
            "preferredcodec": "m4a",
        }
    ],
    "playliststart": int(playlist_start),
    "playlistend": int(playlist_end),
    "outtmpl": f"{str(download_folder)}/%(playlist_index)04d-%(title)s-%(id)s.%(ext)s",
}

# Download youtube videos
if process_type == "youtube":
    !pip install --quiet --root-user-action=ignore yt_dlp
    import yt_dlp
    for url in user_input.split():
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(url)
if process_type == "google_drive":
    !wget --quiet https://github.com/tanaikech/goodls/releases/download/v2.0.3/goodls_linux_amd64 -O download/goodls && chmod +x download/goodls
    for url in user_input.split():
        !cd download && ./goodls -u $url
if process_type == "direct_link":
    !cd download && wget $user_input
if process_type == "curl":
    !cd download && $user_input

# prepare model
if is_faster_whisper:
    !pip install --quiet --root-user-action=ignore "faster-whisper @ https://github.com/SYSTRAN/faster-whisper/archive/refs/heads/master.tar.gz"
    from huggingface_hub.utils import _runtime
    _runtime._is_google_colab = False
    from faster_whisper import WhisperModel
    model = WhisperModel(model, device="cuda", compute_type="float16")
elif is_whisper_jax:
    !pip install --quiet --root-user-action=ignore git+https://github.com/sanchit-gandhi/whisper-jax.git
    import jax.numpy as jnp
    from whisper_jax import FlaxWhisperPipline
    model = FlaxWhisperPipline(
        f"openai/whisper-{model}", dtype=jnp.float16
    )
elif is_insanely_fast_whisper:
    !pip install insanely-fast-whisper
    # !pip install flash-attn --no-build-isolation
    import torch
    from transformers import pipeline
    model = pipeline("automatic-speech-recognition",
                    f"openai/whisper-{model}",
                    torch_dtype=torch.float16,
                    # model_kwargs={"use_flash_attention_2": True},
                    device="cuda:0")
    model.model = model.model.to_bettertransformer()
else:
    !pip install --quiet --root-user-action=ignore git+https://github.com/openai/whisper.git stable-ts
    import whisper
    from stable_whisper import modify_model
    model = whisper.load_model(model)
    # jianfch/stable-ts
    modify_model(model)

# transcribe
for audio_file in sorted(download_folder.iterdir()):
    # Filter out non audio or video files
    mime = mimetypes.guess_type(audio_file)[0]
    if mime is None:
        continue
    mime_type = mime.split("/")[0]
    if mime_type not in ("audio", "video"):
        continue

    print(f"Transcription of {audio_file} will start!")
    text_file = Path(f"{output_folder}/{audio_file.stem}.txt")
    subtitle_file = Path(f"{output_folder}/{audio_file.stem}.srt")

    if is_whisper:
        result = model.transcribe(str(audio_file), language=language)
        # save TXT
        with open(str(text_file), "w", encoding="utf-8") as txt:
            for segment in result["segments"]:
                txt.write(segment["text"].strip() + new_line_separator)
        # save SRT
        # with open(f"{download_folder}/{audio_file.name}.srt", "w", encoding="utf-8") as srt:
        #    whisper.write_srt(result["segments"], file=srt)
        result.to_srt_vtt(str(subtitle_file))
        ## Write into a text file
        # Path(f"{download_folder}/{file_name.name}.txt").write_text(result["text"])
    else:
        if is_faster_whisper:
            segments, info = model.transcribe(
                str(audio_file), language=language, beam_size=5
            )
            print(
                "Detected language '%s' with probability %f"
                % (info.language, info.language_probability)
            )
            segments_list = []
            for segment in segments:
                segments_list.append(segment)
        if is_whisper_jax:
            segments = model(
                str(audio_file), language=language, return_timestamps=True
            )["chunks"]
        if is_insanely_fast_whisper:
            segments = model(str(audio_file), chunk_length_s=30, batch_size=24, return_timestamps=True, generate_kwargs={"language": language},)["chunks"]
        if is_whisper_jax or is_insanely_fast_whisper:
          segments_list = [
                {
                    "start": segment["timestamp"][0],
                    "end": segment["timestamp"][1],
                    "text": segment["text"],
                }
                for segment in segments
            ]
        with open(str(text_file), "w", encoding="utf-8") as txt, open(
            str(subtitle_file), "w", encoding="utf-8"
        ) as srt:
            write_srt(srt, segments_list)
            for segment in segments_list:
                txt.write(segment["text"] if isinstance(segment, dict) else segment.text + new_line_separator)
    files.download(str(text_file))
    files.download(str(subtitle_file))
    audio_file.unlink()
    print("Done!")


     - 16.7 MB 25.6 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done!
